load and normalize all datasets

In [1]:
# Cleveland and Statlog already aligned using 14 standard columns
import pandas as pd
import numpy as np

df_cleveland = pd.read_csv('processed.cleveland.csv', header=None)
df_statlog = pd.read_csv('heart.csv', header=None)
# Framingham
df_fram = pd.read_csv("framingham.csv")
df_fram.rename(columns={
    'sysBP': 'trestbps',
    'glucose': 'chol',
    'TenYearCHD': 'target'
}, inplace=True)
df_fram['target'] = df_fram['target'].astype(int)
df_fram = df_fram[['age', 'sex', 'trestbps', 'chol', 'fbs', 'target']]  # select common subset

# Cardiovascular
df_cardio = pd.read_csv("cardio_train.csv", sep=';')
df_cardio['age'] = (df_cardio['age'] / 365).astype(int)
df_cardio.rename(columns={
    'ap_hi': 'trestbps',
    'cholesterol': 'chol',
    'gluc': 'fbs',
    'cardio': 'target'
}, inplace=True)
df_cardio = df_cardio[['age', 'sex', 'trestbps', 'chol', 'fbs', 'target']]


KeyError: "['sex', 'fbs'] not in index"

In [ ]:
df_cleveland['target'] = df_cleveland['target'].apply(lambda x: 1 if int(x) > 0 else 0)
df_statlog['target'] = df_statlog['target'].apply(lambda x: 1 if int(x) > 0 else 0)


In [ ]:
unified_cols = ['age', 'sex', 'trestbps', 'chol', 'fbs', 'target']

df_cleveland = df_cleveland[unified_cols]
df_statlog = df_statlog[unified_cols]

# Combine all
df_combined = pd.concat([df_cleveland, df_statlog, df_fram, df_cardio], ignore_index=True)
df_combined.dropna(inplace=True)
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

print("Final shape:", df_combined.shape)


In [ ]:
X = df_combined.drop('target', axis=1)
y = df_combined['target']

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
